In [5]:
import socket, ssl

class RawEmailSender:
    def __init__(self, smtp_host, smtp_port=25):
        self.server = (smtp_host, smtp_port)

    def send(self, from_addr, to_addr, subject, body):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect(self.server)
        recv = sock.recv(1024).decode()
        if not recv.startswith('220'): raise Exception("SMTP connect failed")

        def cmd(c): sock.send((c+"\r\n").encode()); return sock.recv(1024).decode()

        cmd(f"EHLO {socket.gethostname()}")
        cmd(f"MAIL FROM:<{from_addr}>")
        cmd(f"RCPT TO:<{to_addr}>")
        cmd("DATA")
        msg = f"Subject: {subject}\r\n\r\n{body}\r\n."
        sock.send((msg+"\r\n").encode())
        cmd("QUIT")
        sock.close()


In [4]:
import webbrowser, time
import urllib.parse

class RawWhatsAppSender:
    def send(self, phone, message):
        url = ("whatsapp://send?phone=" + phone +
               "&text=" + urllib.parse.quote(message))
        webbrowser.open(url)
        time.sleep(10) 

In [2]:
from urllib.parse import urlencode
import http.client
import json

class RawInstagramSender:
    def __init__(self, session_cookies):
        self.cookies = session_cookies

    def send(self, user_id, message):
        conn = http.client.HTTPSConnection("i.instagram.com")
        payload = urlencode({
            'recipient_users': f'[[{user_id}]]',
            'action': 'send_item',
            'client_context': str(int(time.time()))
        })
        payload = payload + '&text=' + urllib.parse.quote(message)
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Cookie': self.cookies
        }
        conn.request("POST", "/api/v1/direct_v2/threads/broadcast/text/", payload, headers)
        res = conn.getresponse()
        return res.status, res.read().decode()


In [3]:
class MultiMessenger(RawEmailSender, RawWhatsAppSender, RawInstagramSender):
    def __init__(self, smtp_host, session_cookies):
        RawEmailSender.__init__(self, smtp_host)
        RawWhatsAppSender.__init__(self)
        RawInstagramSender.__init__(self, session_cookies)



NameError: name 'RawWhatsAppSender' is not defined

In [ ]:


def main():
    smtp_host = 'smtp.example.com' 
    insta_cookies = 'sessionid=ABC123; csrftoken=XYZ789;'  
    
 
    bot = MultiMessenger(smtp_host=smtp_host, session_cookies=insta_cookies)
    
   
    try:
        bot.send(
            from_addr='khwahishsingh2005@gmail.com',
            to_addr='khwahishsingh2005@gmail.com',
            subject='Hello from Python',
            body='This is a test email sent using raw SMTP sockets.'
        )
        print("Email sent successfully")
    except Exception as e:
        print(" Failed to send email:", e)
    
    
    bot.send_whatsapp(phone='+917424988589', message='Hello via WhatsApp link!')
    print(" WhatsApp opened — please click 'Send' manually.")
    
  
    status, response = bot.send(user_id='khwahish5632', message='Hello via raw Instagram DM!')
    if status == 200:
        print(" Instagram DM sent successfully")
    else:
        print(f" Failed to send Instagram DM (status {status}): {response}")

if __name__ == "__main__":
    main()
